In [3]:
import pandas as pd

df = pd.read_csv('final_balanced.csv')

df.head()


/var/folders/m8/6zt09fhx313d5dk_d202076m0000gn/T/ipykernel_51093/2534520098.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('final_balanced.csv')


,crash_date,crash_time,collision_id,borough_traffic,weather_borough,tavg,tmin,tmax,prcp,snow,...,nearest_intersection_name,nearest_intersection_lat,nearest_intersection_lon,nearest_intersection_x_ny,nearest_intersection_y_ny,distance_to_intersection_km,is_crash,day_of_week,month,is_weekend
0,2023-11-01,1:29,4675373.0,BROOKLYN,Brooklyn,8.9,3.9,11.1,0.0,0.0,...,AVE K & OCEAN PKWY,40.621792,-73.970019,9.925730e+05,165813.570502,8.269925e-08,1,2,11,0
1,2021-09-11,9:35,4456314.0,BROOKLYN,Brooklyn,21.1,13.9,27.2,0.0,0.0,...,AUTUMN AVE & LORING AVE,40.667256,-73.866074,1.021403e+06,182404.308244,5.714103e-06,1,5,9,1
2,2021-12-14,8:13,4486609.0,BROOKLYN,Brooklyn,8.7,3.3,13.3,0.0,0.0,...,DECATUR ST & DISTRICT BOUNDARY & SARATOGA AVE,40.683303,-73.917266,1.007196e+06,188233.049282,1.024554e-07,1,1,12,0
3,2021-12-14,17:05,4486555.0,NaN,Brooklyn,8.7,3.3,13.3,0.0,0.0,...,BROOKLYN QUEENS EXPY & S 5 ST,40.708876,-73.956891,9.962017e+05,197541.964310,5.433776e-06,1,1,12,0
4,2021-12-14,8:17,4486660.0,BRONX,Bronx,8.5,3.2,12.0,0.0,0.0,...,BARTOW AVE & BAYCHESTER AVE,40.868771,-73.831733,1.030789e+06,255839.112209,1.117401e-05,1,1,12,0


In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

In [18]:
RANDOM_SEED = 1
np.random.seed(RANDOM_SEED)

### Feature Engineering - Extract Hour

In [19]:
# Convert crash_time to hour
df["hour"] = pd.to_datetime(df["crash_time"], format="%H:%M").dt.hour

### Define numeric columns and categorical columns

In [21]:
num_cols = [
    "hour",
    "day_of_week", "month", "is_weekend",
    "tavg", "tmin", "tmax", "prcp", "snow",
    "wdir", "wspd", "pres",
    "nearest_intersection_lat", "nearest_intersection_lon",
    "distance_to_intersection_km"
]

cat_cols = ["weather_borough"]


### Extract target variable (y)

In [22]:
y = df["is_crash"].astype(int)

### One-Hot Enconde Categorical Columns

In [25]:
ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
X_cat = ohe.fit_transform(df[cat_cols])

# Get the new feature names after encoding
cat_feature_names = ohe.get_feature_names_out(cat_cols)
cat_feature_names

array(['weather_borough_Bronx', 'weather_borough_Brooklyn',
       'weather_borough_Manhattan', 'weather_borough_Queens',
       'weather_borough_Staten Island'], dtype=object)

In [26]:
# Numeric features
X_num = df[num_cols].to_numpy()

# Combine numeric and one-hot encoded features
X = np.hstack([X_num, X_cat])

# Full feature name list
feature_names = num_cols + list(cat_feature_names)


In [29]:
feature_names

['hour',
 'day_of_week',
 'month',
 'is_weekend',
 'tavg',
 'tmin',
 'tmax',
 'prcp',
 'snow',
 'wdir',
 'wspd',
 'pres',
 'nearest_intersection_lat',
 'nearest_intersection_lon',
 'distance_to_intersection_km',
 'weather_borough_Bronx',
 'weather_borough_Brooklyn',
 'weather_borough_Manhattan',
 'weather_borough_Queens',
 'weather_borough_Staten Island']

## Train / Validation / Test

In [30]:
from sklearn.model_selection import train_test_split

# Hold out 20% as your final test set
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y,
    test_size=0.20,
    stratify=y,
    random_state=RANDOM_SEED
)

# Split the remaining 80% into train (60% total) and validation (20% total)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval,
    test_size=0.25,             # 0.25 * 0.80 = 0.20 of original data
    stratify=y_trainval,
    random_state=RANDOM_SEED
)

print("X_train:", X_train.shape)
print("X_val:  ", X_val.shape)
print("X_test: ", X_test.shape)
print("y_train:", y_train.shape)
print("y_val:  ", y_val.shape)
print("y_test: ", y_test.shape)


X_train: (2317560, 20)
X_val:   (772521, 20)
X_test:  (772521, 20)
y_train: (2317560,)
y_val:   (772521,)
y_test:  (772521,)


In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

# 5A) Instantiate the model
rf_params = {
    "n_estimators": 100,      # number of trees (like num_boost_round)
    "max_depth": 6,           # limit tree depth to 6 splits
    "max_features": "sqrt",   # consider √p features at each split
    "random_state": RANDOM_SEED
}

# 2) Instantiate and train
rf = RandomForestClassifier(**rf_params, n_jobs=-1)
rf.fit(X_train, y_train)

# 3) Evaluate AUC
val_probs  = rf.predict_proba(X_val)[:, 1]
test_probs = rf.predict_proba(X_test)[:, 1]
print("Validation AUC =", roc_auc_score(y_val,  val_probs))
print("Test AUC       =", roc_auc_score(y_test, test_probs))


Validation AUC = 0.74345071976419
Test AUC       = 0.7420783779080895


In [34]:
rf_params2 = {
    "n_estimators":    300,       # more trees
    "max_depth":       10,        # deeper trees
    "max_features":    "sqrt",    # keep feature randomness
    "min_samples_leaf": 2,        # prevent tiny leaves
    "random_state":    RANDOM_SEED
}

# 2) Train
rf = RandomForestClassifier(**rf_params2, n_jobs=-1)
rf.fit(X_train, y_train)

# 3) Evaluate
val_probs  = rf.predict_proba(X_val)[:, 1]
test_probs = rf.predict_proba(X_test)[:, 1]
print("Validation AUC =", roc_auc_score(y_val,  val_probs))
print("Test AUC       =", roc_auc_score(y_test, test_probs))


Validation AUC = 0.7647932885315311
Test AUC       = 0.7635456087452193


### Now will use Randomized Search Cross-Validation

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# 1) Define low‑range distributions
param_dist = {
    "n_estimators":    randint(50, 200),    # between 50 and 200 trees
    "max_depth":       randint(3, 10),      # depths 3–10
    "min_samples_leaf":randint(1, 5),       # 1–5 samples per leaf
    "max_features":    ["sqrt", "log2"]     # √p or log2(p)
}

# 2) Set up RandomizedSearchCV
rand_search = RandomizedSearchCV(
    estimator=RandomForestClassifier(
        random_state=RANDOM_SEED,
        n_jobs=-1
    ),
    param_distributions=param_dist,
    n_iter=20,                # try 20 random combos
    scoring="roc_auc",        # optimize AUC
    cv=3,                     # 3‑fold CV on the training set
    random_state=RANDOM_SEED,
    verbose=1
)

# 3) Fit on the TRAIN split
rand_search.fit(X_train, y_train)

# 4) Inspect best results
print("Best CV AUC:", rand_search.best_score_)
print("Best params:", rand_search.best_params_)

# 5) Evaluate on VALIDATION and TEST
best_rf = rand_search.best_estimator_
val_probs  = best_rf.predict_proba(X_val)[:,1]
test_probs = best_rf.predict_proba(X_test)[:,1]
print("Val AUC  =", roc_auc_score(y_val,  val_probs))
print("Test AUC =", roc_auc_score(y_test, test_probs))


Fitting 3 folds for each of 20 candidates, totalling 60 fits
